In [8]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.5 MB/s eta 0:00:00


In [9]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 0.978s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


## Create a streamlit app example


In [55]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
from PIL import Image, ImageOps
import cv2
import numpy as np
import joblib

label_encoder = joblib.load('label_encoder.joblib')

@st.cache(allow_output_mutation=True)

def load_model():
    model = tf.keras.models.load_model('/content/best_model.h5')
    return model

# Load the pre-trained model
with st.spinner('Model is being loaded..'):
     model = load_model()

st.write("# Chicken Disease Image Classification")

file = st.file_uploader("Please upload a Chicken Poop Image", type=["jpg", "png"])

def import_and_predict(image, model):
    # Resize the image to the desired size
    size = (128, 128)
    image = Image.fromarray(image)  # Convert from numpy array to PIL Image
    image = image.resize(size)

    # Convert the image to a numpy array with RGB color
    image_np = np.array(image.convert("RGB"))

    # Make prediction
    prediction = model.predict(np.expand_dims(image_np, axis=0))  # Hapus langkah ini jika tidak diperlukan

    return prediction

    return prediction
def recommend_treatment(disease):
    if disease == "Coccidiosis":
        return """
        - Vaksinasi coccidiosis
        - Perhatikan kondisi dan kebersihan kandang
        - Gunakan Koksidiostat, Ssulfadimetoksin, dan antibiotik (tetrasiklin, eritromisin, spektinomisin, dan tilosin)
        - Lakukan terapi antioksidan
        """
    elif disease == "New Castle Disease":
        return """
        - Ayam yang tertular harus dengan cepat dikarantina
        - Jika terlalu parah, ayam harus dimusnahkan untuk menghindari penularan
        - Vaksinasi melalui tetes mata
        """
    elif disease == "Salmonella":
        return """
        - Desinfeksi kandang dari salmonela
        - Rutin membersihkan tempat makan ayam
        - Berikan Antibiotik seperti amoksisilin dan kolistin
        """
    else:
        return "Rekomendasi pengobatan tidak tersedia."

if file is None:
    st.text("Please upload an image file")
else:
    image = Image.open(file)
    st.image(image, use_column_width=True)

    # Convert the image to a numpy array
    image_np = np.array(ImageOps.exif_transpose(image).convert("RGB"))

    st.write("Input Image Shape:", image_np.shape)

    # Make predictions
    predictions = import_and_predict(image_np, model)
    predicted_class_index = np.argmax(predictions)

    # Convert predicted class index to original class name using label encoder
    predicted_class_name = label_encoder.inverse_transform([predicted_class_index])[0]

    # Display results
    st.write("Prediction:")
    st.write(f"Predicted Class: {predicted_class_name}")
    st.write(f"Confidence: {predictions[0][predicted_class_index]*100:.2f}%")

    # Recommendation based on the predicted class
    treatment_recommendation = recommend_treatment(predicted_class_name)
    st.subheader("Treatment Recommendation:")
    st.write(treatment_recommendation)


Overwriting app.py


## Run streamlit in background

In [56]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

In [ ]:
!npx localtunnel --port 8501